# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [12]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [13]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [14]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
     # Scroll down to the bottom.
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load the page.
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height.
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("到達頁面底端")
            break
        last_height = new_height
        print("網頁更新中...")
    

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [15]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [16]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 金管會今年開罰金融業破3億創新高　保險業挨罰逾1.3億元奪冠

▲金管會公布最新金融業裁罰金額及件數，到今天為止，罰鍰收入達3億122萬元。（圖／記者湯興漢攝）
記者紀佳妘／台北報導
金管會今（26）日公布最新金融業裁罰收入，截至12月26日，對銀行、保險及證券業祭出375張罰單，罰鍰收入達3億122萬元，其中保險業罰鍰收入一口氣來到1億3,560萬元，奪下冠軍寶座。
金管會銀行局表示，今年到12月26日為止，對銀行業開罰23件，罰鍰金額為8,430萬元，與2018全年的24件、4,816萬元相比，顯見罰鍰金額有大幅成長的趨勢。
證券期貨業部分，金管會證期局指出，到今天為止，對證券期貨業開罰286件，裁罰金額達8,132萬元，與去年的239件、6,889萬元相比，裁罰金額也有明顯增加的態勢。
另在保險業方面，金管會保險局說，截至今年12月26為，對保險業開罰件數達66件，超越2018全年的43件，罰鍰收入則來到1億3,560萬元，也比去年一整年的7,520萬元來得高。
金管會今年以來對銀行、證券及保險業違規開罰的罰鍰收入達3億122萬元，其中保險業違規挨罰比例最高，以1億3,560億元奪下冠軍寶座，其中最大一張罰單是南山人壽，因新境界系統之亂挨罰3,000萬元，也是史上最高的罰鍰，而銀行業則因理專或行員涉及挪用客戶資金的案件層出不窮，單是理專挪用資金就開出5,100萬的罰單。
根據金管會資料顯示，今年度編列的違規罰鍰收入（含對上市櫃公司的裁罰）為2.06億元，明年編列的預算為2.02億元。
----------------------------------------------------------------------

[1] 羨煞上班族！玉山銀明年全面加薪　基層員工調薪逾10%

▲玉山銀行今天宣布，明年起全面調薪，其中基層員工加薪幅度達到10%以上。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
玉山銀行今（26）日宣布，自2020年1月1日起全面調薪，全年平均調薪幅度超過8%，其中基層同仁更達10%以上，大手筆的加薪更羨煞不少上班族。
在長期獲利成長的支持下，玉山銀行為了感謝全體主管同仁的付出與努力，以

▲京東物流為中國倉儲業的最大公司之一。（圖／路透社）
記者謝仁傑／綜合報導
知情人士稱，京東物流的估值至少為300億美元（約新台幣9060億元），該部門預計將在明年第二季進行首次公開募股，並將把這筆資金用於倉庫擴張和潛在的收購。
知情人士表示，京東可能於香港或紐約IPO，目前時間表和地點尚未完全確認，且發行量也還有變數。
京東於2017年將物流部門獨立，向第三方公司開放交付和倉儲服務。該部門於2018年從高瓴資本集團（Hillhouse Capital Group）、紅杉資本中國基金（Sequoia Capital China）、騰訊控股（Tencent）、招商局集團和中國人壽等投資者，共籌集了25億美元（約新台幣755億元）的資金。
----------------------------------------------------------------------

[13] 永大明年三箭齊發　總座蔡尚育宣布重啟全球化布局

▲左起為永大財務部經理張瑞薰、發言人林東昇、總經理蔡尚育。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
永大（1507）今（26）日召開法說會，總經理蔡尚育宣布明年將以台灣、大陸及越南市場三箭齊發，並重新啟動全球化布局，包括持續深耕成熟穩定的台灣市場、積極爭取大陸市場訂單業績，還有以越南為中心，拓展東南亞至全球市場，力拚重新回到全球排名前10大品牌。
蔡尚育表示，永大今年前3季營收為99億元，年減12%；稅後淨利7.32億元，年增率23.99%；每股盈餘1.79元，優於去年同期1.44元，毛利率23.97%，並從今年第3季財報看出營收、毛率均有明顯成長，顯示永大營運已經有翻轉的跡象。
蔡尚育指出，明年永大經營策略將以台灣、大陸及越南市場三箭齊發，其中台灣電梯市場從2010年～2018年每年需求1.1萬～1.25萬台，預計今年維持在1.15萬台左右，相對為成熟穩定的市長，因此未來將採取穩中求進的策略，目標提高營收及獲利，預估明年會成長2~3%。
蔡尚育進一步指出，中國大陸市場是全球最大電梯市場，但受到宏觀調控的影響，整體市場從2012年開始急速下降，一直到2017年開始緩步成長，所以雖然去年出貨9000台，但今年已回升至1.9萬～2萬台，並預估明年會有5%的成長，目標積極爭取訂單，期盼營運達到損益兩平。
同時，蔡尚育宣布重新啟動全球化布局

文／謝金河
二○一九年即將走入歷史，最近ＣＮＢＣ電視台針對美國百萬美元以上資產的富豪，進行了一項調查，這項調查顯示，大多數百萬美元富豪不認為二○二○年會有什麼大事發生，或者是美國經濟跌落懸崖，看好與看壞的勢均力敵，其中三九％的受訪者認為美國經濟會走弱，三八％的人認為二○二○年美國經濟仍然會保持強勁，這其中有二七％的人認為二○二○年美國經濟表現會更強。而有五四％的人認為二○二○年美國標普五○○指數至少還會再上漲五％。
不過有趣的是，政黨取向影響了很大的態度，其中民主黨的人認為六二％的比率二○二○年經濟會走弱，看壞經濟的共和黨人只有十九％，四三％的民主黨人認為股市會下跌，而共和黨的人只有十五％，這個結果很有意思，也就是執政的共和黨對美國經濟前景比較樂觀，而在野的民主黨對美國經濟及股市的表現比共和黨悲觀。
這個情況在台灣也是如此，最近韓陣營以林祖嘉為首出來指責台灣經濟過去三年表現是亞洲四小龍之末，三年平均台灣是四小龍之末沒錯，但二○一九年台灣ＧＤＰ成長，百分之百是四小龍之首，就看你用什麼角度來看，但大家要注意的是股市投資人不是看過去舊數據，而是看未來。
感受到台灣經濟谷底爬升
台灣的ＧＤＰ的表現到二○一八年第四季一．七六％觸底，隨之而來的一九年三個季度表現，從首季的一．八三、二．四六，到二．九一％，呈現逐季上揚狀態，第四季預估是三．○四％，是連續五季上升。對照南韓一九年三個季度，一．八、二．一及二％，台灣可說是完勝，再看新加坡三個季度是一．三、○．一及○．一％，香港前三個季度是○．五、○．六及負二．九一％，香港第四季仍然是負值，大家不要憑著主觀意識，看阿拉伯數字也可以感受到台灣經濟谷底爬升的軌跡。
我從來不說台灣經濟有多好，台灣經濟失落三十年了，二○一九年我只用了一個形容詞叫做「漸入佳境」，也就是說，台灣經濟在失落三十年之後，真正開始準備從谷底爬起來，最近股市重返一二○○○點，這只是經濟重上正軌的領先指標。大家可以比較一下，過去三十年，每一年股市結算時，台股與日股漲幅總是敬陪末座。但一九年台股上漲二三．六七％，日股上漲十九．五二％，雙雙高居亞洲前兩位，日本及台灣是過去三十年全球表現最不好的市場，這次翻身格外值得留意。而且大家更要珍惜的是台股這次重新站上一二○○○點，加權指數最高來到一二一二五．九，距離歷史高點一二六八二，只有不到五○○點差距，而日本日經指數天價在三八九五七

▲ 蘋果西打製造商大飲公司前總經理孫幼英等人違證交法被起訴。（圖／記者林敬旻攝）
記者李瑞瑾／綜合報導
知名飲料「蘋果西打」製造商大飲（1213）最大股東、前總經理孫幼英等人涉嫌於民國102至107年間利用假發票交易侵占款項，挪為自己所用等之不法犯行，使大飲對外公告之財報涉有不實，而有違反證券交易法之情事，經臺灣新北地方檢察署提起公訴。
投保中心特別提醒，符合本次求償受理條件之投資人，儘速填妥相關表格、檢附證明文件及簽具相關書件，於109年1月6日前寄回投保中心，才能加入團體訴訟並參與日後訴訟結果所得款項之分配。
----------------------------------------------------------------------

[31] 中火爭議難解　經長不排除親上火線：「解決問題最重要！」

▲台中市政府宣布撤銷2張中火第2及第3號機組燃煤許可證，經濟部長沈榮津表示將支持台電依法提出行政救濟。（圖／記者屠惠剛攝）
記者林淑慧／台北報導
台中市政府昨（25）日宣佈撤銷2張中火第2及第3號機組燃煤許可證，台電回應表示不排除提出行政救濟，經濟部長沈榮津今天重申力挺台電立場，對於是否親上火線與中市府會談，他則強調如果中市府有誠意解決問題，不排除任何可能性，最重要是能解決問題。
台中市政府表示，中火生煤用量已於11月27日超過1104萬公噸上限，並於12月23日再度複查，中火仍繼續超量使用生煤，違規情節重大，對於台電無視法令規定，市府三度開罰，並撤銷2張中火第2及第3號機組燃煤許可證，以捍衛市民健康。
對於中火2部機組燃煤許可證遭撤銷，沈榮津表示，主因在於中市府與台電公司在法令規定的認知有落差，導致中火爭議始終未解，對於經濟部是否出面找中市府溝通，他則表示並未預設立場，如果中市府有誠意要解決問題，經部不排除任何可能性。
另外，針對明年夏季供電備轉容量率是否能達到10%目標，沈榮津表示，經濟部會要求台電務必滿足國人對於穩定供電的期待，已請台電及能源局詳細評估明年夏季供電備轉容量率情況，待結果出爐再對外說明。
經濟部表示，已經嚴格督促台電在兼顧穩定供電下，全力推動降低空污措施，同時也絕對不會讓民眾面臨缺電問題。但仍要呼籲，臺中市政府應該要思考全體國人、台中市民，以及台中在地企業的用電權益，不應做出不合法、不合理的處分。
---------------

▲軟銀與三大銀行融資談判擱淺。（圖／路透社）
記者謝仁傑／綜合報導
知情人士指出，這意味著金援WeWork的95億美元至少今年無法到位。事實上，除了現有貸款額度已達上限的疑慮，瑞穗金融集團（MIZUHO）、三菱日聯金融集團（MUFG）和三井住友金融集團（SMFG）這三大銀行，也擔心挽救這家共享辦公室公司涉及的風險。
據傳這三家銀行正在尋求降低風險，又可提供融資的方式；另一種選擇是用軟銀在中國電商阿里巴巴集團的26％股份作為抵押。
一位資深銀行高層說：「軟銀是一個重要的客戶，我們會想盡辦法去協助，但我們必須考慮信用風險。」
▲美國共享辦公室WeWork。（圖／路透社）
WeWork於10月底宣布將接受軟銀提出的紓困方案，總額約達95億美元，其中包括新增50億美元資金，以及30億美元買斷現有股東股份，並加速原先投資15億美元的計畫。
軟銀在創始人孫正義的領導下，一直賺取豐厚利潤，然而今年不少投資失利，包括WeWork，孫正義也坦承是個錯誤的決策，使軟銀遭到史上最嚴重的打擊。
▲軟銀創辦人孫正義。（圖／路透社）
但孫正義還是認為，撇除Wework創辦人諾伊曼（Adam Neumann）的管理及作為，該公司仍是一間穩固的公司。
----------------------------------------------------------------------

[45] 【名家看經濟】李沃牆／中日韓FTA簽署　台灣不能只靠短期轉單效益

大陸國務院總理李克強於12月24日在成都舉行的「第七屆中日韓工商峰會」指出，三方應確保明年簽署「區域全面經濟夥伴關係協定（RCEP）」，並進一步推動中日韓FTA談判取得進展，日本首相安倍晉三和南韓總統文在寅同聲支持，力爭早日簽署。此外，三國領導人並在會後簽署「未來十年三國合作願景」，具體內容包括守護自由貿易和促進區域經濟一體化、實現技術主導型合作及創新增長、深化環境領域合作及扶持醫療保健產業。
進一步言之，中日韓FTA最早源自於2002年的三國領導人會議，直至2013年3月才在韓國首爾舉行第一輪談判。爾後6年間歷經風風雨雨，談判進度停滯不前；特別是今年7月，日韓二國間爆發貿易爭端，導致中日韓自由貿易區的前景蒙上陰影。11月27日，中日韓三國的自由貿易協定（FTA） 第十六輪談判在首爾舉行。再來就是近日舉行「第七屆中日韓工商峰會」。據

▲南山人壽。（圖／記者湯興漢攝）
財經中心／台北報導
南山人壽這回爆發的新系統之亂，遭金管會重罰3000萬元，並令董事長杜英宗停職2年。金管會主委顧立雄當時直批，從一開始系統更新，無論是風險控管或採購作業一定出問題，「這麼大的核心系統，交給完全沒有保險資訊經驗的德國思愛普軟體（SAP），竟然以這樣的方式進行。」
「連出問題的地方都找不到，每幾天就要接一次客戶反映，投資型保單的金額或單位錯了。」顧立雄說，SAP沒有保單經驗，應從小規模做起、再逐步擴大，每個新系統應作平行測試，且須長時間驗證沒有出問題。
然而，南山人壽卻直接把舊系統整個換掉，把舊資料關閉、停止新資料輸入，也因為舊版系統沒有新資料輸入，導致新舊系統無法平行測試，連顧立雄都搖頭：「現在最大問題，在於大家都傻在那邊，不知道要怎麼改善。」
▲南山人壽前董事長杜英宗（左）、潤泰集團總裁尹衍樑（右）。（合成圖／《ETtoday新聞雲》資料照）
南山人壽原先設想全自動操作系統，現在卻要人工手動檢測，而且發現問題，也無法保證後續不會再出錯，也就是不斷耗費人力檢測，光是人力成本付出就非常龐大，金管會也統計，約有15.2萬張保單永遠檢視不完，而且新問題持續再發生。
「投資型保單」出錯機率高，保戶權益無法確保，因此，金管會認為，南山人壽要先解決「投資型保單系統」，確定不會出問題後，才可恢復辦理投資型保單；再者，核心系統的改善仍有多項待評估。
金管會懲處後，再給南山人壽4個月時間以評估最佳處理方案，且要找第三方單位認證機構，驗證該方案的可行性，再看後續如何做根本性解決，預計年底前要提出改善報告。
▲金管會主委顧立雄。（圖／《ETtoday新聞雲》資料照）
從顧立雄的談話中不難看出，南山人壽面臨問題的嚴重性與急迫性。不過，3個月過去，系統改善情況仍是個謎，第一線業務人員和保戶仍抱怨「境界系統錯誤百出」。
先前，南山人壽董事會推出潤泰集團總裁尹衍樑獨子尹崇堯，代理南山人壽董事長，結果經金管會二次面試後，還是打了回票，人事案緩議。南山人壽又急忙改由南山產險前董事長陳棠代理南山人壽董事長，並將由金管會再次審核。換言之，南山董事會推出尹崇堯的人事佈局，浪費了3個月時間，人事案又得重頭來過。
外界現在最關心的，就是南山能不能確實改善電腦系統問題，在期限內向金管會提報改善計畫，並獲得認可。總不能讓南山人壽的保戶、員工、業務員，無法安心過年，

▲金管會公布壽險最低死亡門檻規定等措施，將於2020年7月1日生效。（圖／pixabay）
記者楊絡懸／台北報導
針對金管會正式公布「人身保險商品審查應注意事項」修正及「人壽保險商品死亡給付對保單價值準備金（保單帳戶價值）之最低比率規範」等措施，進而對保險業商品政策與消費者產生重大影響，壽險公會25日提出4大點說明，這項政策將滿足民眾需求，也發揮保險真正價值，對提升國人保障具有正向助益。
壽險公會說明，第一，這項政策是金管會報行政院「金融發展行動方案」中，有關「促進多元保險保障發揮保險安定社會人心功能」的政策。此次主管機關於推出政策前參採產業意見，並減緩對產業的影響層面與行政作業所需時間等因素綜合考量下，方才頒布相關措施，壽險業將會積極準備與因應此次的修正措施，以維護市場的穩定發展。
第二，台灣預計在2026年邁入「超高齡社會」，伴隨國人的長壽趨勢，為填補退休後收入缺口、降低對晚年生活的衝擊，因此，國人常將兼具保障及穩定累積資產效果的壽險商品，作為退休規劃的理財工具之一。
但根據壽險公會與保發中心2018年統計資料顯示，國人投保率高達249.5%（即國人平均約有2.5張保單），惟每人平均死亡給付僅56.8萬元；若被保險人一旦發生保險事故，對遺族照顧及家庭經濟穩定仍具提升空間。
▲台灣民眾每人平均約持有2.5張保單，但保障額度卻不夠。（圖／pixabay）
「壽險門檻法則」這項新政策實施，希望能確保事故當下能提供適當的保障，避免保險過於偏重理財。壽險業未來保險商品的設計將更貼近國人生命週期的變化，依據人生各階段對保障需求程度設定門檻比率，反倒更能貼近生命週期需要，滿足客戶需求，發揮保險真正價值，對提升國人保障具有正向助益。
第三，「壽險門檻法則」等監理措施的推出，有助引導壽險業者調整商品結構，強化保障型商品等利率敏感度低的保險商品推動，可適度減緩保險公司投資風險及資金去化壓力，朝向強化資產負債管理及提升償付能力的方向邁進，以利未來國際會計準則制度順利接軌，達到企業永續經營的目標。
第四，此次監理政策，雖然對產業的商品政策開發會有很大影響，但壽險業將極盡全力完成此次變更作業，且會對消費者與業務員作出充足的教育訓練與說明，達成主管機關原規劃提高國人保障、穩定商品的利基、強化國際接軌的量能等三贏目的。
【延伸閱讀】
-----------------------------

▲根據金管會統計，今年前10月行動支付總交易金額達880億元，較去年同期大幅成長147%。（示意圖／達志影像）
記者紀佳妘／台北報導
行動支付越來越普及化，根據金管會今（25）日公布，從2016年到今年10月為止，行動支付交易金額達1,520.9億元，其中單是今年前10月行動支付交易金額達880億元，較去年同期大幅成長147%，顯示行動支付市場進入快速成長期。
金管會近年來推動各項行動支付服務，以滿足民眾對金融服務的需求，而所謂的行動支付工具，包括手機綁信用卡（TSM）、手機綁電子票證、行動金融卡、掃碼支付（QR code）、電子支付機構O2O及行動收單機（mPOS）等6項。
根據銀行局統計，今年前10月行動支付總交易金額達880億元，較去年同期的356億元，大幅成長147%。
另觀察近年來行動支付表現，2016年行動支付總交易額達14.9億元，2017年大幅成長至148億元，2018年也一口氣增加至478億元，由此可見近年來皆有跳躍式成長。
金管會主委顧立雄曾表示，行動支付市場進入快速成長期，預估2019全年行動支付總交易額可望破千億元。
金管會2015年喊出電子化支付占民間消費比重，於2020年目標要達到52%，不過到今年上半年為止，僅有41.3%，顧立雄對此指出，確實有一定的難度，但會努力看看。
----------------------------------------------------------------------

[79] 減稅小確幸！基本生活費調高4千元至17.5萬　明年報稅184萬戶受惠

▲政府調高基本生活費為17.5萬元，較去年增加4000元，受惠戶數增加5萬戶。（圖／記者林敬旻攝）
記者吳靜君／台北報導
財政部今（25）日公告108年度每人基本生活所需之費用金額為17.5萬元，較去年增加4000元，民眾明年5月申報綜合所得稅時適用。根據財政部統計，預計有184萬戶惠因此受惠，較前一年（107年、今年5月報稅）增加5萬戶，對民眾報稅來說減稅小確幸又多一些。
財政部表示，有關108年度每人基本生活所需之費用金額，依納保法第4條規定，參照行政院主計總處公布最近一年(107年)每人可支配所得中位數29萬1,514元，按中位數之60%訂定108年度每人基本生活所需費用為17.5萬元，較107年度17.1萬元，增加4,000元。
納稅者權

----------------------------------------------------------------------

[87] 國票金獲利佳、股價便宜！明年現金殖利率有望5%以上

▲國票金控董事長魏啟林。（圖／記者湯興漢攝）
記者吳靜君／台北報導
國票金今（25）日召開法說會，國票金表示，至今年11月底每股淨值為12.49元，但是現在股價大約11元，P／B（股價淨值比）仍低，國票金價格算低。同時，國票金未來股利配發也會以現金優先考量，就過去國票金的股利配發政策來看，明年現金殖利率有望5%以上。
國票金選擇在聖誕節開法說會，根據國票金自結數，今年前11月稅後純益為24.34億元，較去年同期成長37.49%，每股稅後純益（EPS）0.86元。
國票金表示，今年前11月獲利成長的原因，主要是因為子公司國際票券授信業務近年積極調整授信結構，使票券收益成長，並且持續建置外幣債部位，外幣債買賣斷利益及養券收益成長；子公司國票證券因為證券經紀據點固守市占，自營業穩健操作、承銷團隊深耕市場，三大業務引擎俱開，所以獲利大幅優於過往。
國票金自結，今年11月底止，每股淨值12.49元，但是目前國票金目前每股股價只接近11元，股價淨值比低，股價仍算便宜。
就股利政策方面，就過去現金殖利率來說，去年因為獲利較為吃緊，今（2019）年現金股利為0.45元、股票股利為0.1元，現金殖利率僅為4.13%、股利分配率也有七、八成。
但再前兩年2018年與2017年的現金殖利率都有5.71%、5.68%，國票金表示，未來仍以多配發現金股利為優先考量，以今年獲利表現頗佳的狀況來看，明（2020）年現金殖利率也有望5%以上。
至於與日本樂天銀行共同設立純網銀樂天國際商業銀行的進度上，100億元的資金已經到位，等樂天國際商業銀行籌備處收足股款後，續辦公司設立登記及籌設工作，完成後再向金管會申請核發營業執照，預計明年下半年正式營業。其他兩家純網銀連線銀行、將來銀行都已經取得公平會的同意，但是樂天國際商銀卻尚未，國票金解釋，因為樂天在台灣有其他的子公司，在準備財務資料上需要多一點的時間，預計明年初就可以拿到公平會的核可。
----------------------------------------------------------------------

[88] 商業周刊